# Fast Polynomial Interpolation for $2^n - 1$ Elements

Over the field $F_{2^n}$

**TODO: Please complete these things to finish the notebook**
- Proof of correctness for the interpolation
- Write better intuitive explanations of why this works

In [1]:
import galois
import numpy as np

## Intuition

We can flip some of the matrix to find the inverse. The Python code below shows how this works.

```python 
V[1:, 1:] = np.flip(V[1:, 1:], 0)
```

In [2]:
# Set up field parameters
N = 2**3
GF = galois.GF(N, display='power')

# Display both the Vandermonde matrix and its inverse
V = GF.Vandermonde(GF.primitive_element, (N - 1), (N - 1))
print(V, '\n')
print(np.linalg.inv(V))

[[  1,   1,   1,   1,   1,   1,   1],
 [  1,   α, α^2, α^3, α^4, α^5, α^6],
 [  1, α^2, α^4, α^6,   α, α^3, α^5],
 [  1, α^3, α^6, α^2, α^5,   α, α^4],
 [  1, α^4,   α, α^5, α^2, α^6, α^3],
 [  1, α^5, α^3,   α, α^6, α^4, α^2],
 [  1, α^6, α^5, α^4, α^3, α^2,   α]] 

[[  1,   1,   1,   1,   1,   1,   1],
 [  1, α^6, α^5, α^4, α^3, α^2,   α],
 [  1, α^5, α^3,   α, α^6, α^4, α^2],
 [  1, α^4,   α, α^5, α^2, α^6, α^3],
 [  1, α^3, α^6, α^2, α^5,   α, α^4],
 [  1, α^2, α^4, α^6,   α, α^3, α^5],
 [  1,   α, α^2, α^3, α^4, α^5, α^6]]


In [3]:
# Set up field parameters
N = 2**3
GF = galois.GF(N, display='int')

# Display both the Vandermonde matrix and its inverse
V = GF.Vandermonde(GF.primitive_element, (N - 1), (N - 1))
print(V, '\n')
print(np.linalg.inv(V))

[[1, 1, 1, 1, 1, 1, 1],
 [1, 2, 4, 3, 6, 7, 5],
 [1, 4, 6, 5, 2, 3, 7],
 [1, 3, 5, 4, 7, 2, 6],
 [1, 6, 2, 7, 4, 5, 3],
 [1, 7, 3, 2, 5, 6, 4],
 [1, 5, 7, 6, 3, 4, 2]] 

[[1, 1, 1, 1, 1, 1, 1],
 [1, 5, 7, 6, 3, 4, 2],
 [1, 7, 3, 2, 5, 6, 4],
 [1, 6, 2, 7, 4, 5, 3],
 [1, 3, 5, 4, 7, 2, 6],
 [1, 4, 6, 5, 2, 3, 7],
 [1, 2, 4, 3, 6, 7, 5]]


In [4]:
def eval_poly(poly: galois.Poly, x: int):
    # Because the polynomial is generated from a Vandermonde matrix with a primitive element as the base of 
    # the exponent we must evaluate slightly differently.
    return poly(poly.field.primitive_element**x)    

In [5]:
def Vandermonde_inv(GF: galois.GF):
    """
    This function calculates the inverse of the Vandermonde matrix for field GF(2^k) 
    with 2^k - 1 primitive elements used as `x` values in the matrix.

    WARNING: This function is only usable up to ~GF(2^15) as any larger uses significant memory.

    Parameters
    ----------
    k : int
        The order of the field GF(2^k).
    """

    assert GF.characteristic == 2, "GF must be a field of order 2^k"

    # Because the structure of the matrix is understood, we can efficiently calculate the inverse
    # See above for more details.
    rows = cols = GF.order - 1
    V_inv = GF.Vandermonde(GF.primitive_element, rows, cols)
    V_inv[1:, 1:] = np.flip(V_inv[1:, 1:], 0)

    return V_inv

### Example computing $V^{-1}(\bf{x})$

In [6]:
N = 2**3
GF = galois.GF(N)
y = GF.Random(N - 1) # Random values to interpolate

coefficients = y @ Vandermonde_inv(GF)
poly = galois.Poly(np.flip(coefficients))

print(f"Evaluation of {poly} at all x values:")
for i in range(N-1):
    assert y[i] == eval_poly(poly, i)
    print(f"(x={i}, y_interpolated={eval_poly(poly, i)}, y_real={y[i]})")

Evaluation of 5x^6 + 4x^5 + 5x^4 + x^3 + 5x^2 + 6x + 5 at all x values:
(x=0, y_interpolated=3, y_real=3)
(x=1, y_interpolated=6, y_real=6)
(x=2, y_interpolated=6, y_real=6)
(x=3, y_interpolated=1, y_real=1)
(x=4, y_interpolated=5, y_real=5)
(x=5, y_interpolated=7, y_real=7)
(x=6, y_interpolated=5, y_real=5)


In [7]:
def iterative_interpolation(GF: galois.GF, y: galois.FieldArray):
    # Because the structure of the matrix is understood, we can efficiently calculate the inverse
    # See above for more details.

    assert GF.characteristic == 2
    assert len(y) == GF.order - 1

    # Flip first row of the Vandermonde matrix to get the inverse as per the trick above
    V_row_flip = GF.Vandermonde(GF.primitive_element, 2, len(y))[1]
    V_row_flip[1:] = np.flip(V_row_flip[1:], 0)

    # Calculate the coefficients of the polynomial one by one
    # This keeps the memory usage low as we do not need the entire Vandermonde matrix
    coefficients = GF.Zeros(len(y))
    for i in range(len(y)):
        coefficients[i] = V_row_flip**i @ y

    return galois.Poly(np.flip(coefficients))

### Example computing the polynomial coefficients iteratively

In [8]:
N = 2**3
GF = galois.GF(N)
y = GF.Random(N - 1) # Random values to interpolate
poly = iterative_interpolation(GF, y)

print(f"Evaluation of {poly} at all x values:")
for i in range(N-1):
    assert y[i] == eval_poly(poly, i)
    print(f"(x={i}, y_interpolated={eval_poly(poly, i)}, y_real={y[i]})")

Evaluation of 2x^6 + 3x^5 + 3x^4 + 5x^3 + 2x^2 + 3x + 6 at all x values:
(x=0, y_interpolated=0, y_real=0)
(x=1, y_interpolated=5, y_real=5)
(x=2, y_interpolated=7, y_real=7)
(x=3, y_interpolated=3, y_real=3)
(x=4, y_interpolated=0, y_real=0)
(x=5, y_interpolated=5, y_real=5)
(x=6, y_interpolated=2, y_real=2)


In [9]:
def inplace_interpolation(GF: galois.GF, y: galois.FieldArray):
    assert GF.characteristic == 2
    assert len(y) == GF.order - 1

    # Calculate the coefficients of the polynomial one by one
    # This keeps the memory usage low as we do not need the entire Vandermonde matrix
    N = len(y)
    coefficients = GF.Zeros(N)
    for i in range(N):
        for j in range(N):
            position = (i*j) % N
            coefficients[i] += (GF.primitive_element)**(N - position) * y[j]

    return galois.Poly(np.flip(coefficients))


N = 2**3
GF = galois.GF(N)
y = GF.Random(N - 1) # Random values to interpolate
poly = inplace_interpolation(GF, y)

print(f"Evaluation of {poly} at all x values:")
for i in range(N-1):
    assert y[i] == eval_poly(poly, i)
    print(f"(x={i}, y_interpolated={eval_poly(poly, i)}, y_real={y[i]})")

Evaluation of 4x^6 + 4x^5 + 2x^4 + x^3 + 6x^2 + 4 at all x values:
(x=0, y_interpolated=1, y_real=1)
(x=1, y_interpolated=6, y_real=6)
(x=2, y_interpolated=1, y_real=1)
(x=3, y_interpolated=0, y_real=0)
(x=4, y_interpolated=2, y_real=2)
(x=5, y_interpolated=5, y_real=5)
(x=6, y_interpolated=5, y_real=5)
